In [62]:
!pip3 install requests

Python(13569) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip3 install --upgrade pip



### 1. Import required Libraries

In [63]:
import requests
import sqlite3
from datetime import datetime
from urllib.parse import urlparse, parse_qs


### 2. All constants used in notebook

In [64]:
base_url = "https://de-demo-api.adtriba.app/"
endpoint = "v1/api/data" 
api_key = "woope1Pei5zieg"
from_date = "2021-08-01" 
to_date = "2021-09-30"    
db_file_name ="fashion_brand_logs.db"


### 3. Fetch Tracking Logs: Defines a function to fetch tracking logs from an API using the provided API details and query parameters. It handles the API response and prints the fetched logs.


In [65]:
# API details

headers = {
    "x-api-key": api_key
}

# Function to fetch tracking logs
def fetch_tracking_logs(from_date=None, to_date=None, offset=0, limit=10000):
    params = {
        "offset": offset,
        "limit": limit
    }

    if from_date:
        params["from_date"] = from_date
    if to_date:
        params["to_date"] = to_date

    
    response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

    # Check for successful request
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

logs = fetch_tracking_logs(from_date=from_date, to_date=to_date, offset=0, limit=100000)

if logs:
    print("Tracking Logs:")
    print(logs[:10])
else:
    print("No logs fetched.")


Tracking Logs:
[{'log_id': '000302cb-4aaf-4154-b6b2-912d5e42cfff', 'cookie_id': '65b5dfea-a8a0-4723-9fdb-08e43feba62b', 'referrer_domain': None, 'location': 'https://www.fashion-brand.com/', 'log_ts': '2021-08-20T11:24:09.698000', 'action': 'pageview'}, {'log_id': '000d32ba-f26b-486f-9ff8-f844866a4924', 'cookie_id': '44319efe-66a9-4c9d-8e2c-8fc0a45c77f3', 'referrer_domain': 'info.fashion-brand.com', 'location': 'https://www.fashion-brand.com/?hsLang=de', 'log_ts': '2021-09-29T09:47:45.797000', 'action': 'pageview'}, {'log_id': '0014735e-b457-4ac3-bc2d-5270e156efb1', 'cookie_id': '8517de02-01c0-462c-b11d-b4963d358b24', 'referrer_domain': 'www.google.com.hk', 'location': 'https://blog.fashion-brand.com/en/unified-marketing-measurement-why-umm-is-the-next-generation-of-marketing-attribution', 'log_ts': '2021-08-25T05:48:12.559000', 'action': 'pageview'}, {'log_id': '001729c6-4d7d-4f41-b184-e7d5123937b7', 'cookie_id': 'f9f2d6f0-9a0c-4cb8-8431-a5ca18e3e1fe', 'referrer_domain': 'm.youtube.co


### 4. Parse Query Parameters from Logs : Defines a function to parse query parameters from the fetched logs and prints the parsed logs.

In [66]:
def parse_logs(logs):
    parsed_logs = []
    for log in logs:
        parsed_logs.append(
            (
                log.get("log_id"),
                log.get("cookie_id"),
                log.get("location"),
                log.get("referrer_domain"),
                str(log.get("log_ts")),
                log.get("action"),
            )
        )
    return parsed_logs

parsed_logs = parse_logs(logs)
#print(parsed_logs[:10])


[('000302cb-4aaf-4154-b6b2-912d5e42cfff', '65b5dfea-a8a0-4723-9fdb-08e43feba62b', 'https://www.fashion-brand.com/', None, '2021-08-20T11:24:09.698000', 'pageview'), ('000d32ba-f26b-486f-9ff8-f844866a4924', '44319efe-66a9-4c9d-8e2c-8fc0a45c77f3', 'https://www.fashion-brand.com/?hsLang=de', 'info.fashion-brand.com', '2021-09-29T09:47:45.797000', 'pageview'), ('0014735e-b457-4ac3-bc2d-5270e156efb1', '8517de02-01c0-462c-b11d-b4963d358b24', 'https://blog.fashion-brand.com/en/unified-marketing-measurement-why-umm-is-the-next-generation-of-marketing-attribution', 'www.google.com.hk', '2021-08-25T05:48:12.559000', 'pageview'), ('001729c6-4d7d-4f41-b184-e7d5123937b7', 'f9f2d6f0-9a0c-4cb8-8431-a5ca18e3e1fe', 'https://www.fashion-brand.com/de?utm_term=&utm_campaign=DE_Remarketing+Display+Nov+2019&utm_source=adwords&utm_medium=ppc&hsa_acc=8061560483&hsa_cam=8140702782&hsa_grp=124518466552&hsa_ad=529526218787&hsa_src=d&hsa_tgt=&hsa_kw=&hsa_mt=&hsa_net=adwords&hsa', 'm.youtube.com', '2021-08-16T22:5


### 5. Store Logs in SQLite Database : Stores the parsed logs in an SQLite database by creating a table and inserting the data into it.

In [67]:

# Ensure the connection is closed properly to avoid database lock issues
try:
    conn = sqlite3.connect(db_file_name)
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS pageviews (
        log_id TEXT,
        cookie_id TEXT,
        location TEXT,
        referrer_domain TEXT,
        log_ts TEXT,
        action TEXT
    )
    ''')
    cursor.execute('DELETE FROM pageviews')
    
    cursor.executemany('''
    INSERT INTO pageviews (log_id, cookie_id, location, referrer_domain, log_ts, action)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', parsed_logs)

   
    conn.commit()

    #cursor.execute('SELECT * FROM pageviews limit 10')
    #rows = cursor.fetchall()

    #for row in rows:
        #print(row)
finally:
    conn.close()


### 6. Extract Query Parameters and Apply Channel Mapping Rules : Defines a function to extract query parameters from the logs and apply channel mapping rules to categorize the logs into different channels such as "Organic Search", "Social", "Email", "Paid Search", "Referral", and "Direct".

In [68]:
def extract_query_params(logs):
    extracted_data = []
    for log in logs:
        url = log['location']
        referrer_domain = log['referrer_domain']
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        
        utm_source = query_params.get('utm_source', [None])[0]
        utm_campaign = query_params.get('utm_campaign', [None])[0]
        utm_medium = query_params.get('utm_medium', [None])[0]
        gclid = query_params.get('gclid', [None])[0]
        
        channel = "Unknown"
        
        # Channel Mapping Rules
        if utm_medium == 'organic':
            channel = "Organic Search"
        elif utm_medium and any(prefix in utm_medium.lower() for prefix in ['social', 'sm']):
            channel = "Social"
        elif utm_source and utm_source.lower() in ['facebook', 'instagram', 'ig', 'fb', 'linkedin']:
            channel = "Social"
        elif utm_medium == 'email':
            channel = "Email"
        elif utm_medium and any(keyword in utm_medium.lower() for keyword in ['cpc', 'ppc', 'paid', 'paidsearch']):
            channel = "Paid Search"
        elif gclid:
            channel = "Paid Search"
        elif utm_source and any(keyword in utm_source.lower() for keyword in ['google', 'yahoo']):
            channel = "Paid Search"
        elif not (utm_source or utm_campaign or utm_medium) and referrer_domain:
            channel = "Referral"
        elif not (utm_source or utm_campaign or utm_medium or referrer_domain):
            channel = "Direct"
        
        extracted_data.append((utm_source, utm_campaign, utm_medium, gclid, channel))
    
    return extracted_data

extracted_data = extract_query_params(logs)
#print(extracted_data[:5])


[(None, None, None, None, 'Direct'), (None, None, None, None, 'Referral'), (None, None, None, None, 'Referral'), ('adwords', 'DE_Remarketing Display Nov 2019', 'ppc', None, 'Paid Search'), ('adwords', 'DE_Remarketing Display Nov 2019', 'ppc', None, 'Paid Search')]



### 7. Store Extracted Data in SQLite Database : Stores the extracted data in an SQLite database by creating a new dimension table called `channel` and inserting the data into it.


In [69]:

conn = sqlite3.connect(db_file_name)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS channel (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        utm_source TEXT,
        utm_campaign TEXT,
        utm_medium TEXT,
        gclid TEXT,
        channel TEXT
    )
''')
cursor.execute('DELETE FROM channel')

cursor.executemany('''
    INSERT INTO channel (utm_source, utm_campaign, utm_medium, gclid, channel)
    VALUES (?, ?, ?, ?, ?)
''', extracted_data)

conn.commit()
conn.close()


### 8. Create and Query a View: Creates a view to filter out pageviews where the referrer contains the domain "fashion-brand.com" and count the number of pageviews per channel. 


In [70]:

conn = sqlite3.connect(db_file_name)
cursor = conn.cursor()

cursor.execute('DROP VIEW IF EXISTS filtered_pageviews')

cursor.execute('''
    CREATE VIEW filtered_pageviews AS
    SELECT COUNT(*) as pageview_count, c.channel FROM pageviews p, channel c 
    WHERE p.action = 'pageview' AND p.referrer_domain LIKE '%fashion-brand.com%'
    GROUP BY c.channel
''')

cursor.execute('SELECT * FROM filtered_pageviews')
filtered_logs = cursor.fetchall()

print("Pageviews per Channel:")
for log in filtered_logs:
    print(f"{log[0]}: {log[1]}")

conn.close()


Pageviews per Channel:
2949756: Direct
14436: Email
3669952: Paid Search
5240268: Referral
117092: Social
1604: Unknown
